# Installing and importing packages

In [ ]:
! pip install deepl

In [ ]:
!pip install youtube-transcript-api

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
from google.colab import files
from tqdm.notebook import tqdm
import time

tqdm.pandas()

In [ ]:
from googleapiclient.discovery import build

from youtube_transcript_api import YouTubeTranscriptApi

In [ ]:
# API key was removed

In [ ]:
youtube = build("youtube", "v3", developerKey=api_key)

# Channel ID finder

In [ ]:
CHANNEL_NAMES = ["@latinanoticias", "@americanoticiaspe","@LaRepublica"]


# Defining function that fetches channel IDs
def get_channel_ids(api_key, channel_names):

    base_url = "https://www.googleapis.com/youtube/v3/search"
    results = {}

    for name in channel_names:
        params = {
            "part": "snippet",
            "q": name,
            "type": "channel",
            "key": api_key
        }

        response = requests.get(base_url, params=params).json()

        if "items" in response and response["items"]:
            channel_id = response["items"][0]["id"]["channelId"]
            results[name] = channel_id
        else:
            results[name] = "Not Found"

    return results



# Fetch Channel IDs
channel_ids = get_channel_ids(api_key, CHANNEL_NAMES)

for name, channel_id in channel_ids.items():
    print(f"{name} - Channel ID: {channel_id}")


# Collecting videos IDs for Latina Noticias





## Full month periods

In [ ]:
#Dataframe with full month periods

# Define the start and end dates
start_date = datetime(2019, 2, 1)
end_date = datetime(2025, 3, 1)

data = []

# Generate the start and end date-times for each full month period
current_date = start_date
while current_date < end_date:
    start_date_str = current_date.strftime("%Y-%m-%dT%H:%M:%S-05:00")

    next_month = current_date.replace(day=28) + timedelta(days=4)
    end_date_str = next_month.replace(day=1) - timedelta(seconds=1)
    end_date_str = end_date_str.strftime("%Y-%m-%dT%H:%M:%S-05:00")

    data.append([start_date_str, end_date_str])

    # Move to the first day of the next month
    current_date = next_month.replace(day=1)

# Create a df
date_ranges = pd.DataFrame(data, columns=["start", "end"])

date_ranges


In [ ]:
# Initial test
channel_id = "UCpSJ5fGhmAME9Kx2D3ZvN3Q"

# Defined keywords
keyword = "robo|robó|roban|robaron|delincuentes|criminales|asalto|asaltó|asaltan|asaltaron|matan|mató|mataron|ladrón|asesinato|asesinado|asesinan|asesinaron|extorsionadores|extorsionador|extorsionan|extorsionaron|extorsionó|secuestro|secuestradores|secuestrada|secuestrado|secuestró|secuestraron|balacera|balean|balearon|balazos|sicariato|mafias|mafia|sicarios"

In [ ]:
# Function used for the first sample
# Function for getting video info for one channel id, one keyword, one time frame
# Returns dictionary of all videos found, and prints number

def get_youtube_videos(api_key, channel_id, keyword, max_results=50, order="date",
                       published_after=None, published_before=None):

    videos = []
    # Prepare API request parameters
    params = {
        "part": "snippet",
        "channelId": channel_id,
        "q": keyword,
        "type": "video",
        "maxResults": max_results,
        "order": order,
    }

    if published_after:
        params["publishedAfter"] = published_after

    if published_before:
        params["publishedBefore"] = published_before

    # Execute API request
    request = youtube.search().list(**params)
    response = request.execute()

    # Extract video details
    for item in response.get("items", []):
        video_id = item["id"]["videoId"]
        video_title = item["snippet"]["title"]
        video_url = f"https://www.youtube.com/watch?v={video_id}"
        video_date = item["snippet"]["publishedAt"]

        videos.append({
            "video_id": video_id,
            "title": video_title,
            "url": video_url,
            "channel": channel_id,
            "keyword": keyword,
            "video_date": video_date})

    print("Number of videos found between " + published_after + " and " + published_before + ": " + str(len(videos)))

    return videos

In [ ]:
# First saample creation
# Create a for loop that goes through each time frame
    # gets all youtube videos and store them in a dataframe

channel_id = "UCpSJ5fGhmAME9Kx2D3ZvN3Q" # Latina Noticias
max_results = 50
order = "date"
keyword = "robo|robó|roban|robaron|delincuentes|criminales|asalto|asaltó|asaltan|asaltaron|matan|mató|mataron|ladrón|asesinato|asesinado|asesinan|asesinaron|extorsionadores|extorsionador|extorsionan|extorsionaron|extorsionó|secuestro|secuestradores|secuestrada|secuestrado|secuestró|secuestraron|balacera|balean|balearon|balazos|sicariato|mafias|mafia|sicarios"

# Create empty dataframe for first time running it
df = pd.DataFrame(columns=['video_id','title','url','channel','keyword','video_date'])

for index, row in date_ranges.iterrows():
    published_after = row['start']
    published_before = row['end']

    df_new = pd.DataFrame.from_records(get_youtube_videos(api_key, channel_id, keyword, max_results,
                                                          order, published_after, published_before))
    df = pd.concat([df, df_new], ignore_index=True)

In [ ]:
df.shape

In [ ]:
# Save df as csv
df.to_csv("video_titles.csv")

In [ ]:
# new function
#Function for getting video info for one channel id, one keyword, one time frame
# Returns dictionary of all videos found and number of videos, and prints number
#

def get_youtube_videos(api_key, channel_id, keyword, max_results=50, order="date",
                       published_after=None, published_before=None):

    videos = []

    # Prepare API request parameters
    params = {
        "part": "snippet",
        "channelId": channel_id,
        "q": keyword,
        "type": "video",
        "maxResults": max_results,
        "order": order,
    }

    if published_after:
        params["publishedAfter"] = published_after

    if published_before:
        params["publishedBefore"] = published_before

    # Execute API request
    request = youtube.search().list(**params)
    response = request.execute()

    # Extract video details
    for item in response.get("items", []):
        video_id = item["id"]["videoId"]
        video_title = item["snippet"]["title"]
        video_url = f"https://www.youtube.com/watch?v={video_id}"
        video_date = item["snippet"]["publishedAt"]

        videos.append({
            "video_id": video_id,
            "title": video_title,
            "url": video_url,
            "channel": channel_id,
            "keyword": keyword,
            "video_date": video_date})


    print("Number of videos found between " + published_after + " and " + published_before + ": " + str(len(videos)))



    return videos, len(videos)

In [ ]:
# applying new function
# Create a for loop that goes through each time frame
    # gets all youtube videos and store them in a dataframe

channel_id = "UCpSJ5fGhmAME9Kx2D3ZvN3Q" # Latina Noticias
max_results = 50
order = "date"
keyword = "robo|robó|roban|robaron|delincuentes|criminales|asalto|asaltó|asaltan|asaltaron|matan|mató|mataron|ladrón|asesinato|asesinado|asesinan|asesinaron|extorsionadores|extorsionador|extorsionan|extorsionaron|extorsionó|secuestro|secuestradores|secuestrada|secuestrado|secuestró|secuestraron|balacera|balean|balearon|balazos|sicariato|mafias|mafia|sicarios"

# Create empty dataframe for first time running it
df = pd.DataFrame(columns=['video_id','title','url','channel','keyword','video_date'])
videos_list = []
videos_count_list = []

for index, row in date_ranges.iterrows():
    published_after = row['start']
    published_before = row['end']


    all = get_youtube_videos(api_key, channel_id, keyword, max_results,
                                                          order, published_after, published_before)
    videos = all[0]
    video_count = all[1]
    videos_list.append(videos)
    videos_count_list.append(video_count)


In [ ]:
flattened_list = np.concatenate(videos_list).tolist()
df1 = pd.DataFrame(flattened_list)
df1.drop(columns=['keywords'], inplace=True)

In [ ]:
df1.to_csv("videos.csv")
files.download('videos.csv')

In [ ]:
date_ranges['videos_count'] = videos_count_list

In [ ]:
date_ranges.to_csv("date_ranges.csv")
files.download('date_ranges.csv')

In [ ]:
uploaded = files.upload()

In [ ]:
df2 = pd.read_csv("video_titles_03.08.csv")
df1 = pd.read_csv("video_titles.csv")
date_ranges_df = pd.read_csv("date_ranges.csv")

In [ ]:
# checking which videos are overlaping in the two dfs

df1_only = df1[~df1['video_id'].isin(df2['video_id'])]
df1_only

In [ ]:
df2_only = df2[~df2['video_id'].isin(df1['video_id'])]
df2_only

In [ ]:
date_ranges_df.drop(columns=['Unnamed: 0'], inplace=True)
date_ranges_df

## Half-month periods

In [ ]:
#creating a new df where the time ranges of those that have vidoe count == 50 are split in half

# Convert start and end columns to datetime
date_ranges_df["start"] = pd.to_datetime(date_ranges_df["start"])
date_ranges_df["end"] = pd.to_datetime(date_ranges_df["end"])

df_filtered = date_ranges_df[date_ranges_df["videos_count"] == 50].copy()

# Create new DataFrame for half-month periods
split_periods = []

for _, row in df_filtered.iterrows():
    start_date = row["start"]
    mid_date = start_date + timedelta(days=15)  # Mid-month
    end_date = row["end"]

    # Ensure mid_date does not exceed end_date
    if mid_date > end_date:
        mid_date = end_date

    # First half
    split_periods.append({"start": start_date, "end": mid_date - timedelta(seconds=1)})

    # Second half
    split_periods.append({"start": mid_date, "end": end_date})

# Create new DataFrame with split periods
df_half_months = pd.DataFrame(split_periods)

# Convert back to the required format
df_half_months["start"] = df_half_months["start"].dt.strftime("%Y-%m-%dT%H:%M:%S-05:00")
df_half_months["end"] = df_half_months["end"].dt.strftime("%Y-%m-%dT%H:%M:%S-05:00")

# Display result
print(df_half_months)

In [ ]:
# Getting the youtube videos for the split periods
channel_id = "UCpSJ5fGhmAME9Kx2D3ZvN3Q" # Latina Noticias
max_results = 50
order = "date"
keyword = "robo|robó|roban|robaron|delincuentes|criminales|asalto|asaltó|asaltan|asaltaron|matan|mató|mataron|ladrón|asesinato|asesinado|asesinan|asesinaron|extorsionadores|extorsionador|extorsionan|extorsionaron|extorsionó|secuestro|secuestradores|secuestrada|secuestrado|secuestró|secuestraron|balacera|balean|balearon|balazos|sicariato|mafias|mafia|sicarios"

# Create empty dataframe for first time running it
df = pd.DataFrame(columns=['video_id','title','url','channel','keyword','video_date'])
videos_list = []
videos_count_list = []

for index, row in df_half_months.iterrows():
    published_after = row['start']
    published_before = row['end']


    all = get_youtube_videos(api_key, channel_id, keyword, max_results,
                                                          order, published_after, published_before)
    videos = all[0]
    video_count = all[1]
    videos_list.append(videos)
    videos_count_list.append(video_count)

In [ ]:
#creating a df with the outputs

flattened_list = np.concatenate(videos_list).tolist()
df3 = pd.DataFrame(flattened_list)
df3.drop(columns=['keyword'], inplace=True)

In [ ]:
#adding the video counts to the date ranges table

df_half_months['videos_count'] = videos_count_list

In [ ]:
df3.to_csv("video_titles_03.09.csv")
files.download('video_titles_03.09.csv')

In [ ]:
df_half_months.to_csv("date_ranges_2.csv")
files.download('date_ranges_2.csv')

In [ ]:
df_half_months

In [ ]:
count50 = df_half_months[df_half_months["videos_count"] == 50]
count50.shape

## Week periods

In [ ]:
#splitting to weekly periods
split_periods = []

df_filtered = df_half_months[df_half_months["videos_count"] == 50].copy()

# Create new DataFrame for half-month periods
split_periods = []

for _, row in df_filtered.iterrows():
    start_date = row["start"]
    mid_date = start_date + timedelta(days=7)
    end_date = row["end"]

    # Ensure mid_date does not exceed end_date
    if mid_date > end_date:
        mid_date = end_date

    # First half
    split_periods.append({"start": start_date, "end": mid_date - timedelta(seconds=1)})

    # Second half
    split_periods.append({"start": mid_date, "end": end_date})

# Create new df
df_weeks = pd.DataFrame(split_periods)

# required format
df_weeks["start"] = df_weeks["start"].dt.strftime("%Y-%m-%dT%H:%M:%S-05:00")
df_weeks["end"] = df_weeks["end"].dt.strftime("%Y-%m-%dT%H:%M:%S-05:00")

print(df_weeks)

In [ ]:
# Getting the youtube videos for the weekly periods
channel_id = "UCpSJ5fGhmAME9Kx2D3ZvN3Q" # Latina Noticias
max_results = 50
order = "date"
keyword = "robo|robó|roban|robaron|delincuentes|criminales|asalto|asaltó|asaltan|asaltaron|matan|mató|mataron|ladrón|asesinato|asesinado|asesinan|asesinaron|extorsionadores|extorsionador|extorsionan|extorsionaron|extorsionó|secuestro|secuestradores|secuestrada|secuestrado|secuestró|secuestraron|balacera|balean|balearon|balazos|sicariato|mafias|mafia|sicarios"

# Create empty dataframe for first time running it
df = pd.DataFrame(columns=['video_id','title','url','channel','keyword','video_date'])
videos_list = []
videos_count_list = []

for index, row in df_weeks.iterrows():
    published_after = row['start']
    published_before = row['end']


    all = get_youtube_videos(api_key, channel_id, keyword, max_results,
                                                          order, published_after, published_before)
    videos = all[0]
    video_count = all[1]
    videos_list.append(videos)
    videos_count_list.append(video_count)

In [ ]:
# creating a df with the outputs
flattened_list = np.concatenate(videos_list).tolist()
df4 = pd.DataFrame(flattened_list)
df4.drop(columns=['keyword'], inplace=True)

In [ ]:
# adding the video counts to the date ranges table
df_weeks['videos_count'] = videos_count_list

In [ ]:
df_weeks.to_csv("date_ranges_3.csv")
files.download('date_ranges_3.csv')
df4.to_csv("video_titles_03.09.2.csv")
files.download('video_titles_03.09.2.csv')

## Half-week periods

In [ ]:
#splitting to half-weekly periods
split_periods = []

df_filtered = df_weeks[df_weeks["videos_count"] == 50].copy()

# Convert 'start' and 'end' to datetime
df_filtered["start"] = pd.to_datetime(df_filtered["start"])
df_filtered["end"] = pd.to_datetime(df_filtered["end"])

# Create new DataFrame for half-month periods
split_periods = []

for _, row in df_filtered.iterrows():
    start_date = row["start"]
    mid_date = start_date + timedelta(days=4)  # Mid-month
    end_date = row["end"]

    # Ensure mid_date does not exceed end_date
    if mid_date > end_date:
        mid_date = end_date

    # First half
    split_periods.append({"start": start_date, "end": mid_date - timedelta(seconds=1)})

    # Second half
    split_periods.append({"start": mid_date, "end": end_date})

# Create new df
df_half_weeks = pd.DataFrame(split_periods)

# Convert back to the required format
df_half_weeks["start"] = df_half_weeks["start"].dt.strftime("%Y-%m-%dT%H:%M:%S-05:00")
df_half_weeks["end"] = df_half_weeks["end"].dt.strftime("%Y-%m-%dT%H:%M:%S-05:00")

print(df_half_weeks)

In [ ]:
count50 = df_weeks[df_weeks["videos_count"] == 50]
count50.shape

In [ ]:
# Getting the youtube videos for the half weekly periods
channel_id = "UCpSJ5fGhmAME9Kx2D3ZvN3Q" # Latina Noticias
max_results = 50
order = "date"
keyword = "robo|robó|roban|robaron|delincuentes|criminales|asalto|asaltó|asaltan|asaltaron|matan|mató|mataron|ladrón|asesinato|asesinado|asesinan|asesinaron|extorsionadores|extorsionador|extorsionan|extorsionaron|extorsionó|secuestro|secuestradores|secuestrada|secuestrado|secuestró|secuestraron|balacera|balean|balearon|balazos|sicariato|mafias|mafia|sicarios"

# Create empty dataframe for first time running it
df = pd.DataFrame(columns=['video_id','title','url','channel','keyword','video_date'])
videos_list = []
videos_count_list = []

for index, row in df_half_weeks.iterrows():
    published_after = row['start']
    published_before = row['end']


    all = get_youtube_videos(api_key, channel_id, keyword, max_results,
                                                          order, published_after, published_before)
    videos = all[0]
    video_count = all[1]
    videos_list.append(videos)
    videos_count_list.append(video_count)

In [ ]:
#creating a df with the outputs
flattened_list = np.concatenate(videos_list).tolist()
df5 = pd.DataFrame(flattened_list)
df5.drop(columns=['keyword'], inplace=True)

In [ ]:
#adding the video counts to the date ranges table
df_half_weeks['videos_count'] = videos_count_list

In [ ]:
df_half_weeks.to_csv("date_ranges_4.csv")
files.download('date_ranges_4.csv')
#df5.to_csv("video_titles_03.09.3.csv")
#files.download('video_titles_03.09.3.csv')

## Couple days periods

In [ ]:
uploaded = files.upload()

In [ ]:
df_half_weeks = pd.read_csv("date_ranges_4.csv")

In [ ]:
#splitting to half-weekly periods
split_periods = []

df_filtered = df_half_weeks[df_half_weeks["videos_count"] == 50].copy()

# Convert 'start' and 'end' to datetime
df_filtered["start"] = pd.to_datetime(df_filtered["start"])
df_filtered["end"] = pd.to_datetime(df_filtered["end"])

# Create new DataFrame for half-month periods
split_periods = []

for _, row in df_filtered.iterrows():
    start_date = row["start"]
    mid_date = start_date + timedelta(days=2)
    end_date = row["end"]

    # Ensure mid_date does not exceed end_date
    if mid_date > end_date:
        mid_date = end_date

    # First half
    split_periods.append({"start": start_date, "end": mid_date - timedelta(seconds=1)})

    # Second half
    split_periods.append({"start": mid_date, "end": end_date})

# Create new df
df_couple_days = pd.DataFrame(split_periods)

# Convert back to the required format
df_couple_days["start"] = df_couple_days["start"].dt.strftime("%Y-%m-%dT%H:%M:%S-05:00")
df_couple_days["end"] = df_couple_days["end"].dt.strftime("%Y-%m-%dT%H:%M:%S-05:00")

print(df_couple_days)

In [ ]:
#validating that the number of periods id correct
count50 = df_half_weeks[df_half_weeks["videos_count"] == 50]
count50.shape

In [ ]:
# Getting the youtube videos for the half weekly periods
channel_id = "UCpSJ5fGhmAME9Kx2D3ZvN3Q" # Latina Noticias
max_results = 50
order = "date"
keyword = "robo|robó|roban|robaron|delincuentes|criminales|asalto|asaltó|asaltan|asaltaron|matan|mató|mataron|ladrón|asesinato|asesinado|asesinan|asesinaron|extorsionadores|extorsionador|extorsionan|extorsionaron|extorsionó|secuestro|secuestradores|secuestrada|secuestrado|secuestró|secuestraron|balacera|balean|balearon|balazos|sicariato|mafias|mafia|sicarios"

# Create empty dataframe for first time running it
df = pd.DataFrame(columns=['video_id','title','url','channel','keyword','video_date'])
videos_list = []
videos_count_list = []

for index, row in df_couple_days.iterrows():
    published_after = row['start']
    published_before = row['end']


    all = get_youtube_videos(api_key, channel_id, keyword, max_results,
                                                          order, published_after, published_before)
    videos = all[0]
    video_count = all[1]
    videos_list.append(videos)
    videos_count_list.append(video_count)

In [ ]:
#creating a df with the outputs
flattened_list = np.concatenate(videos_list).tolist()
df6 = pd.DataFrame(flattened_list)
df6.drop(columns=['keyword'], inplace=True)

In [ ]:
#adding the video counts to the date ranges table
df_couple_days['videos_count'] = videos_count_list

In [ ]:
df_couple_days.to_csv("date_ranges_5.csv")
files.download('date_ranges_5.csv')
df6.to_csv("video_titles_03.09.4.csv")
files.download('video_titles_03.09.4.csv')

## Combining all the video that have not already been collected

In [ ]:
uploaded = files.upload()

In [ ]:
df1 = pd.read_csv("video_titles.csv")
df2 = pd.read_csv("video_titles_03.09.csv")
df3 = pd.read_csv("video_titles_03.09.2.csv")
df4 = pd.read_csv("video_titles_03.09.3.csv")
df5 = pd.read_csv("video_titles_03.09.4.csv")

In [ ]:
combined_df = pd.concat([df2, df3, df4, df5])
combined_df= combined_df.drop(columns=['Unnamed: 0'])

In [ ]:
#removing duplicates
combined_df = combined_df.drop_duplicates(subset="video_id", keep="first")
combined_df = combined_df.reset_index(drop=True)
combined_df

In [ ]:
combined_df.to_csv("combined_df_L.csv")
files.download('combined_df_L.csv')

In [ ]:
combined_test = pd.concat([df1,combined_df])
combined_test = combined_test.drop_duplicates(subset="video_id", keep="first")
combined_test = combined_test.reset_index(drop=True)

In [ ]:
combined_test

#Collecting video IDs for LaRepublica

## Full month periods

In [ ]:
#df with full month periods

# Define the start and end dates
start_date = datetime(2019, 2, 1)
end_date = datetime(2025, 3, 1)

# Create a list to hold the data
data = []

# Generate the start and end date-times for each full month period
current_date = start_date
while current_date < end_date:
    # cet the start date of the current month
    start_date_str = current_date.strftime("%Y-%m-%dT%H:%M:%S-05:00")

    # calculate the next month's first day for the end date
    next_month = current_date.replace(day=28) + timedelta(days=4)  # this gives us the next month
    end_date_str = next_month.replace(day=1) - timedelta(seconds=1)
    end_date_str = end_date_str.strftime("%Y-%m-%dT%H:%M:%S-05:00")

    # Append to the data list
    data.append([start_date_str, end_date_str])

    # Move to the first day of the next month
    current_date = next_month.replace(day=1)

# Create a DataFrame
date_ranges = pd.DataFrame(data, columns=["start", "end"])

date_ranges


In [ ]:
#Function for getting video info for one channel id, one keyword, one time frame
# Returns dictionary of all videos found and number of videos, and prints number
#

def get_youtube_videos(api_key, channel_id, keyword, max_results=50, order="date",
                       published_after=None, published_before=None):

    videos = []

    # Prepare API request parameters
    params = {
        "part": "snippet",
        "channelId": channel_id,
        "q": keyword,
        "type": "video",
        "maxResults": max_results,
        "order": order,
    }

    if published_after:
        params["publishedAfter"] = published_after

    if published_before:
        params["publishedBefore"] = published_before

    # Execute API request
    request = youtube.search().list(**params)
    response = request.execute()

    # Extract video details
    for item in response.get("items", []):
        video_id = item["id"]["videoId"]
        video_title = item["snippet"]["title"]
        video_url = f"https://www.youtube.com/watch?v={video_id}"
        video_date = item["snippet"]["publishedAt"]

        videos.append({
            "video_id": video_id,
            "title": video_title,
            "url": video_url,
            "channel": channel_id,
            "keyword": keyword,
            "video_date": video_date})


    print("Number of videos found between " + published_after + " and " + published_before + ": " + str(len(videos)))



    return videos, len(videos)

In [ ]:
# applying new function
# Create a for loop that goes through each time frame
    # gets all youtube videos and store them in a dataframe

channel_id = "UC-B7Xv56uNRDkj0vC3QW8Cg" # LaRepublica
max_results = 50
order = "date"
keyword = "robo|robó|roban|robaron|delincuentes|criminales|asalto|asaltó|asaltan|asaltaron|matan|mató|mataron|ladrón|asesinato|asesinado|asesinan|asesinaron|extorsionadores|extorsionador|extorsionan|extorsionaron|extorsionó|secuestro|secuestradores|secuestrada|secuestrado|secuestró|secuestraron|balacera|balean|balearon|balazos|sicariato|mafias|mafia|sicarios"

# Create empty dataframe for first time running it
df = pd.DataFrame(columns=['video_id','title','url','channel','keyword','video_date'])
videos_list = []
videos_count_list = []

for index, row in date_ranges.iterrows():
    published_after = row['start']
    published_before = row['end']


    all = get_youtube_videos(api_key, channel_id, keyword, max_results,
                                                          order, published_after, published_before)
    videos = all[0]
    video_count = all[1]
    videos_list.append(videos)
    videos_count_list.append(video_count)


In [ ]:
# another attempt on 20.03
# Create a for loop that goes through each time frame
    # gets all youtube videos and store them in a dataframe

channel_id = "UC-B7Xv56uNRDkj0vC3QW8Cg" # LaRepublica
max_results = 50
order = "date"
keyword = "robo|robó|roban|robaron|delincuentes|criminales|asalto|asaltó|asaltan|asaltaron|matan|mató|mataron|ladrón|asesinato|asesinado|asesinan|asesinaron|extorsionadores|extorsionador|extorsionan|extorsionaron|extorsionó|secuestro|secuestradores|secuestrada|secuestrado|secuestró|secuestraron|balacera|balean|balearon|balazos|sicariato|mafias|mafia|sicarios"

# Create empty dataframe for first time running it
df = pd.DataFrame(columns=['video_id','title','url','channel','keyword','video_date'])
videos_list = []
videos_count_list = []

for index, row in date_ranges.iterrows():
    published_after = row['start']
    published_before = row['end']


    all = get_youtube_videos(api_key, channel_id, keyword, max_results,
                                                          order, published_after, published_before)
    videos = all[0]
    video_count = all[1]
    videos_list.append(videos)
    videos_count_list.append(video_count)


In [ ]:
flattened_list = np.concatenate(videos_list).tolist()
df_R_1 = pd.DataFrame(flattened_list)
df_R_1.drop(columns=['keyword'], inplace=True)

date_ranges['videos_count'] = videos_count_list

In [ ]:
df_R_1.to_csv("videos_titles_R1.csv")
files.download('videos_titles_R1.csv')
date_ranges.to_csv("date_ranges_R1.csv")
files.download('date_ranges_R1.csv')

In [ ]:
uploaded = files.upload()

In [ ]:
date_ranges = pd.read_csv("date_ranges_R1.csv")

## Week periods

In [ ]:
#splitting to weekly periods
split_periods = []

df_filtered = date_ranges[date_ranges["videos_count"] == 50].copy()


# Convert to datetime format
df_filtered["start"] = pd.to_datetime(df_filtered["start"])
df_filtered["end"] = pd.to_datetime(df_filtered["end"])

# Create a new list to store weekly periods
weekly_periods = []

# Loop through each full-month period and split into weeks
for _, row in df_filtered.iterrows():
    start_date = row["start"]
    end_date = row["end"]

    current_date = start_date

    while current_date < end_date:
        next_week = current_date + timedelta(days=7)
        if next_week > end_date:
            next_week = end_date  # Ensure the last week aligns with the month's end

        weekly_periods.append({
            "start": current_date,
            "end": next_week - timedelta(seconds=1)  # End at 23:59:59 of the previous day
        })

        current_date = next_week  # Move to the next week

# Create new DataFrame for weekly periods
week_ranges = pd.DataFrame(weekly_periods)

# Convert datetime format back to string
week_ranges["start"] = week_ranges["start"].dt.strftime("%Y-%m-%dT%H:%M:%S-05:00")
week_ranges["end"] = week_ranges["end"].dt.strftime("%Y-%m-%dT%H:%M:%S-05:00")

# Display the result
print(week_ranges)

In [ ]:
#Going through week_ranges
# Create a for loop that goes through each time frame
    # gets all youtube videos and store them in a dataframe

channel_id = "UCPhm2I2wk4vqjENwhn3px8A" # América Noticias
max_results = 50
order = "date"
keyword = "robo|robó|roban|robaron|delincuentes|criminales|asalto|asaltó|asaltan|asaltaron|matan|mató|mataron|ladrón|asesinato|asesinado|asesinan|asesinaron|extorsionadores|extorsionador|extorsionan|extorsionaron|extorsionó|secuestro|secuestradores|secuestrada|secuestrado|secuestró|secuestraron|balacera|balean|balearon|balazos|sicariato|mafias|mafia|sicarios"

# Create empty dataframe for first time running it
df = pd.DataFrame(columns=['video_id','title','url','channel','keyword','video_date'])
videos_list = []
videos_count_list = []

for index, row in week_ranges.iterrows():
    published_after = row['start']
    published_before = row['end']


    all = get_youtube_videos(api_key, channel_id, keyword, max_results,
                                                          order, published_after, published_before)
    videos = all[0]
    video_count = all[1]
    videos_list.append(videos)
    videos_count_list.append(video_count)


In [ ]:
flattened_list = np.concatenate(videos_list).tolist()
df_R_2 = pd.DataFrame(flattened_list)
df_R_2.drop(columns=['keyword'], inplace=True)

week_ranges['videos_count'] = videos_count_list

In [ ]:
df_R_2.to_csv("videos_titles_R2.csv")
files.download('videos_titles_R2.csv')
week_ranges.to_csv("date_ranges_R2.csv")
files.download('date_ranges_R2.csv')

# Collecting videos IDs for América Noticias

## Full month periods

In [ ]:
# applying new function
# Create a for loop that goes through each time frame
    # gets all youtube videos and store them in a dataframe

channel_id = "UCPhm2I2wk4vqjENwhn3px8A" # América Noticias
max_results = 50
order = "date"
keyword = "robo|robó|roban|robaron|delincuentes|criminales|asalto|asaltó|asaltan|asaltaron|matan|mató|mataron|ladrón|asesinato|asesinado|asesinan|asesinaron|extorsionadores|extorsionador|extorsionan|extorsionaron|extorsionó|secuestro|secuestradores|secuestrada|secuestrado|secuestró|secuestraron|balacera|balean|balearon|balazos|sicariato|mafias|mafia|sicarios"

# Create empty dataframe for first time running it
df = pd.DataFrame(columns=['video_id','title','url','channel','keyword','video_date'])
videos_list = []
videos_count_list = []

for index, row in date_ranges.iterrows():
    published_after = row['start']
    published_before = row['end']


    all = get_youtube_videos(api_key, channel_id, keyword, max_results,
                                                          order, published_after, published_before)
    videos = all[0]
    video_count = all[1]
    videos_list.append(videos)
    videos_count_list.append(video_count)


In [ ]:
flattened_list = np.concatenate(videos_list).tolist()
df_A_1 = pd.DataFrame(flattened_list)
df_A_1.drop(columns=['keyword'], inplace=True)

date_ranges['videos_count2'] = videos_count_list

In [ ]:
df_A_1.to_csv("videos_titles_A1.csv")
files.download('videos_titles_A1.csv')
#date_ranges.to_csv("date_ranges_A1.csv")
#files.download('date_ranges_A1.csv')

## Week periods

In [ ]:
uploaded = files.upload()

In [ ]:
date_ranges = pd.read_csv("date_ranges_A1.csv")

In [ ]:
#splitting to weekly periods
split_periods = []

df_filtered = date_ranges[date_ranges["videos_count"] == 50].copy()


# Convert to datetime format
df_filtered["start"] = pd.to_datetime(df_filtered["start"])
df_filtered["end"] = pd.to_datetime(df_filtered["end"])

# Create a new list to store weekly periods
weekly_periods = []

# Loop through each full-month period and split into weeks
for _, row in df_filtered.iterrows():
    start_date = row["start"]
    end_date = row["end"]

    current_date = start_date

    while current_date < end_date:
        next_week = current_date + timedelta(days=7)
        if next_week > end_date:
            next_week = end_date  # Ensure the last week aligns with the month's end

        weekly_periods.append({
            "start": current_date,
            "end": next_week - timedelta(seconds=1)  # End at 23:59:59 of the previous day
        })

        current_date = next_week  # Move to the next week

# Create new DataFrame for weekly periods
week_ranges = pd.DataFrame(weekly_periods)

# Convert datetime format back to string
week_ranges["start"] = week_ranges["start"].dt.strftime("%Y-%m-%dT%H:%M:%S-05:00")
week_ranges["end"] = week_ranges["end"].dt.strftime("%Y-%m-%dT%H:%M:%S-05:00")

print(week_ranges)

In [ ]:
## split the df in half as it has too many rows

split_index = len(week_ranges) // 2

week_ranges1 = week_ranges.iloc[:split_index]
week_ranges2 = week_ranges.iloc[split_index:]

print("First DataFrame:")
print(week_ranges1)
print("\nSecond DataFrame:")
print(week_ranges2)

In [ ]:
#Going through week_ranges1
# Create a for loop that goes through each time frame
    # gets all youtube videos and store them in a dataframe

channel_id = "UCPhm2I2wk4vqjENwhn3px8A" # América Noticias
max_results = 50
order = "date"
keyword = "robo|robó|roban|robaron|delincuentes|criminales|asalto|asaltó|asaltan|asaltaron|matan|mató|mataron|ladrón|asesinato|asesinado|asesinan|asesinaron|extorsionadores|extorsionador|extorsionan|extorsionaron|extorsionó|secuestro|secuestradores|secuestrada|secuestrado|secuestró|secuestraron|balacera|balean|balearon|balazos|sicariato|mafias|mafia|sicarios"

# Create empty dataframe for first time running it
df = pd.DataFrame(columns=['video_id','title','url','channel','keyword','video_date'])
videos_list = []
videos_count_list = []

for index, row in week_ranges1.iterrows():
    published_after = row['start']
    published_before = row['end']


    all = get_youtube_videos(api_key, channel_id, keyword, max_results,
                                                          order, published_after, published_before)
    videos = all[0]
    video_count = all[1]
    videos_list.append(videos)
    videos_count_list.append(video_count)


In [ ]:
flattened_list = np.concatenate(videos_list).tolist()
df_A_2 = pd.DataFrame(flattened_list)
df_A_2.drop(columns=['keyword'], inplace=True)

week_ranges1['videos_count'] = videos_count_list

In [ ]:
df_A_2.to_csv("videos_titles_A2.csv")
files.download('videos_titles_A2.csv')
week_ranges1.to_csv("date_ranges_A2.csv")
files.download('date_ranges_A2.csv')

In [ ]:
#Going through week_ranges2
# Create a for loop that goes through each time frame
    # gets all youtube videos and store them in a dataframe

channel_id = "UCPhm2I2wk4vqjENwhn3px8A" # América Noticias
max_results = 50
order = "date"
keyword = "robo|robó|roban|robaron|delincuentes|criminales|asalto|asaltó|asaltan|asaltaron|matan|mató|mataron|ladrón|asesinato|asesinado|asesinan|asesinaron|extorsionadores|extorsionador|extorsionan|extorsionaron|extorsionó|secuestro|secuestradores|secuestrada|secuestrado|secuestró|secuestraron|balacera|balean|balearon|balazos|sicariato|mafias|mafia|sicarios"

# Create empty dataframe for first time running it
df = pd.DataFrame(columns=['video_id','title','url','channel','keyword','video_date'])
videos_list = []
videos_count_list = []

for index, row in week_ranges2.iterrows():
    published_after = row['start']
    published_before = row['end']


    all = get_youtube_videos(api_key, channel_id, keyword, max_results,
                                                          order, published_after, published_before)
    videos = all[0]
    video_count = all[1]
    videos_list.append(videos)
    videos_count_list.append(video_count)

In [ ]:
flattened_list = np.concatenate(videos_list).tolist()
df_A_3 = pd.DataFrame(flattened_list)
df_A_3.drop(columns=['keyword'], inplace=True)

week_ranges2['videos_count'] = videos_count_list

In [ ]:
df_A_3.to_csv("videos_titles_A3.csv")
files.download('videos_titles_A3.csv')
week_ranges2.to_csv("date_ranges_A3.csv")
files.download('date_ranges_A3.csv')

# Combining all videos

In [ ]:
uploaded = files.upload()

In [ ]:
df1 = pd.read_csv("videos_titles_R1.csv")
df2 = pd.read_csv("videos_titles_R2.csv")
df3 = pd.read_csv("videos_titles_A1 (1).csv")
df4 = pd.read_csv("videos_titles_A2.csv")
df5 = pd.read_csv("videos_titles_A3.csv")
df6 = pd.read_csv("videos_titles_A1(probably a mistake).csv")
df7 = pd.read_csv("combined_df_L.csv")

In [ ]:
#combining all, except LN
combined_df = pd.concat([df1, df2, df3, df4, df5, df6])
combined_df= combined_df.drop(columns=['Unnamed: 0'])
combined_df

In [ ]:
#removing duplicates
combined_df = combined_df.drop_duplicates(subset="video_id", keep="first")
combined_df = combined_df.reset_index(drop=True)
combined_df

# Transripts

## Initial sample

In [ ]:
# for test
video_id = # Video ID removed

In [ ]:
# without timestamp

def get_transcript(video_id, language='es'):
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=[language])
        return [line['text'] for line in transcript]

    except Exception as e:
        print("Error:", e)

In [ ]:
test_transcript = get_transcript(video_id)

In [ ]:
# Go through dataframe and request transcripts for each video

# Doesnt consume youtube api quota
# find a way to count errors (seems like only 130 videos didnt get transcripts)

df['transcript'] = df['video_id'].apply(get_transcript)

In [ ]:
df.tail()

In [ ]:
# Save df as csv
df.to_csv("video_titles.csv")

In [ ]:
test_transcript

In [ ]:
#removing 'sounds'

cleaned_lines = [line for line in test_transcript if not line.startswith("[")]

#merging into a single text block
clean_transcript = " ".join(cleaned_lines)

## Secondary sample

In [ ]:
# trnascript function that stores also the error messages

def get_transcript(video_id, language='es'):
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=[language])
        return " ".join([line['text'] for line in transcript])  # Join transcript into a single string
    except Exception as e:
        return f"Error: {e}"  # Return error message

In [ ]:
# getting transcripts for videos from Latina Noticias
df7['transcript'] = df7['video_id'].progress_apply(get_transcript)

In [ ]:
df7= df7.drop(columns=['Unnamed: 0'])
df7.to_csv("transcripts_L.csv")
files.download('transcripts_L.csv')

In [ ]:
# getting transcripts for videos from AN and La Republica
combined_df['transcript'] = combined_df['video_id'].progress_apply(get_transcript)

In [ ]:
combined_df.to_csv("transcripts_R&A.csv")
files.download('transcripts_R&A.csv')

In [ ]:
uploaded = files.upload()

In [ ]:
df1 = pd.read_csv("transcripts_R&A.csv")
df2 = pd.read_csv("transcripts_L.csv")
df3 = pd.read_csv("video_titles.csv")

In [ ]:
combined_df = pd.concat([df1, df2, df3])
combined_df= combined_df.drop(columns=['Unnamed: 0'])
combined_df.drop(columns=['keyword'], inplace=True)

In [ ]:
#removing duplicates
combined_df = combined_df.drop_duplicates(subset="video_id", keep="first")
combined_df = combined_df.reset_index(drop=True)
combined_df

In [ ]:
combined_df.to_csv("all_videos_transcript.csv")
files.download('all_videos_transcript.csv')

In [ ]:
# Filter out rows where the transcript contains an error message
df_errors = combined_df[combined_df['transcript'].str.contains("Error:", na=False)].copy()

In [ ]:
# function for fetching the data inc. the errors
def get_transcript(video_id, language='es'):
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=[language])
        return " ".join([line['text'] for line in transcript])  # Convert list to string
    except Exception as e:
        return f"Error: {e}"

In [ ]:
#testing on a smaller sample
dfs = [df_errors.iloc[i:i + 100].copy() for i in range(0, len(df_errors), 100)]
df_test = dfs[0]

In [ ]:
# Attempt to retrieve transcripts for failed cases
for index, row in tqdm(df_test.iterrows(), total=df_test.shape[0], desc="Retrying transcripts"):
    time.sleep(5)

    df_test.at[index, 'new_transcript'] = get_transcript(row['video_id'])

In [ ]:
error_count = df_test[df_test['new_transcript'].str.startswith("Error", na=False)].shape[0]
print(f"Total transcripts with errors: {error_count}")
error_count = df_test[df_test['new_transcript'].str.contains('age-restricted', case=False, na=False)].shape[0]
print(f"Total transcripts with age-restricted errors: {error_count}")
error_count = df_test[df_test['new_transcript'].str.contains('Subtitles are disabled', case=False, na=False)].shape[0]
print(f"Total transcripts with subtitles are disabled errors: {error_count}")
error_count = df_test[df_test['new_transcript'].str.contains('video is unplayable', case=False, na=False)].shape[0]
print(f"Total transcripts with video is unplayable errors: {error_count}")
error_count = df_test[df_test['new_transcript'].str.contains('requested language codes:', case=False, na=False)].shape[0]
print(f"Total transcripts with requested language codes: ['es'] errors: {error_count}")

In [ ]:
df_test.iloc[32]['new_transcript']

In [ ]:
#removing age restricted errors
df_age_restricted = df_errors[df_errors['transcript'].str.contains('age-restricted', case=False, na=False)]

# Remove the age-restricted rows from the original df_errors
df_errors2 = df_errors[~df_errors['transcript'].str.contains('age-restricted', case=False, na=False)]

In [ ]:
#removing Subtitles are disabled errors
df_sub_disabled = df_errors2[df_errors2['transcript'].str.contains('Subtitles are disabled', case=False, na=False)]

# Remove the rows from the original df_errors
df_errors3 = df_errors2[~df_errors2['transcript'].str.contains('Subtitles are disabled', case=False, na=False)]

In [ ]:
#removing video is unplayable errors
df_unplayable = df_errors3[df_errors3['transcript'].str.contains('video is unplayable', case=False, na=False)]

# Remove the rows from the original df_errors
df_errors4 = df_errors3[~df_errors3['transcript'].str.contains('video is unplayable', case=False, na=False)]

In [ ]:
#removing video is requested language codes: errors
df_language_code = df_errors4[df_errors4['transcript'].str.contains('requested language codes:', case=False, na=False)]

# Remove the rows from the original df_errors
df_errors5 = df_errors4[~df_errors4['transcript'].str.contains('requested language codes:', case=False, na=False)]

In [ ]:
#removing video is IP errors
df_IP = df_errors5[df_errors5['transcript'].str.contains('YouTube is blocking requests from your IP', case=False, na=False)]

# Remove the rows from the original df_errors
df_errors6 = df_errors5[~df_errors5['transcript'].str.contains('YouTube is blocking requests from your IP', case=False, na=False)]

In [ ]:
print(df_age_restricted.shape,df_sub_disabled.shape,df_unplayable.shape,df_errors4.shape,df_language_code.shape,df_IP.shape,df_errors6.shape)

In [ ]:
df_errors5.tail(5)

In [ ]:
# Attempt to retrieve transcripts for first 1000 failed cases

for index, row in tqdm(df_IP.iloc[:1000].iterrows(), total=1000, desc="Retrying transcripts"):
    #time.sleep(5)
    df_IP.at[index, 'new_transcript'] = get_transcript(row['video_id'])

In [ ]:
# running the function through all the rows in batches

for index, row in tqdm(df_IP.iloc[7000:].iterrows(), total=1000, desc="Retrying transcripts"):
    time.sleep(3)
    df_IP.at[index, 'new_transcript'] = get_transcript(row['video_id'])

In [ ]:
x = df_IP.iloc[7000:]
error_count = x[x['new_transcript'].str.startswith("Error", na=False)].shape[0]
print(f"Total transcripts with errors: {error_count}")
error_count = df_IP[df_IP['new_transcript'].str.startswith("Error", na=False)].shape[0]
print(f"Total transcripts with errors: {error_count}")

In [ ]:
df_IP.to_csv("IP_errors3.csv")
files.download('IP_errors3.csv')

In [ ]:
uploaded = files.upload()

In [ ]:
df_IP = pd.read_csv("IP_errors3.csv")

In [ ]:
all_transcripts = pd.read_csv("all_videos_transcript.csv")
all_transcripts_old = pd.read_csv("all_videos_transcript.csv")

In [ ]:
df_IP = df_IP.drop(columns=['transcript'])
df_IP = df_IP.rename(columns={'new_transcript': 'transcript'})  # Rename column

In [ ]:
# Merging the two dataframes
all_transcripts.update(df_IP)

In [ ]:
all_transcripts.shape

In [ ]:
error_count = all_transcripts[all_transcripts['transcript'].str.startswith("Error", na=False)].shape[0]
print(f"Total transcripts with errors: {error_count}")
error_count = df_IP[df_IP['transcript'].str.startswith("Error", na=False)].shape[0]
print(f"Total transcripts with errors: {error_count}")

In [ ]:
error_count = all_transcripts[all_transcripts['transcript'].str.startswith("Error", na=False)].shape[0]
print(f"Total transcripts with errors: {error_count}")
error_count = all_transcripts[all_transcripts['transcript'].str.contains('age-restricted', case=False, na=False)].shape[0]
print(f"Total transcripts with age-restricted errors: {error_count}")
error_count = all_transcripts[all_transcripts['transcript'].str.contains('Subtitles are disabled', case=False, na=False)].shape[0]
print(f"Total transcripts with subtitles are disabled errors: {error_count}")
error_count = all_transcripts[all_transcripts['transcript'].str.contains('video is unplayable', case=False, na=False)].shape[0]
print(f"Total transcripts with video is unplayable errors: {error_count}")
error_count = all_transcripts[all_transcripts['transcript'].str.contains('requested language codes:', case=False, na=False)].shape[0]
print(f"Total transcripts with requested language codes: ['es'] errors: {error_count}")
error_count = all_transcripts[all_transcripts['transcript'].str.contains('YouTube is blocking requests from your IP', case=False, na=False)].shape[0]
print(f"Total transcripts with IP errors: {error_count}")


In [ ]:
# Merging the two dataframes
all_transcripts.update(df_IP)

In [ ]:
all_transcripts.to_csv("all_transcripts2.csv")
files.download('all_transcripts2.csv')

In [ ]:
uploaded = files.upload()

In [ ]:
all_transcripts = pd.read_csv("/all_transcripts2.csv")

In [ ]:
# separating the errors
df_IP2 = all_transcripts[all_transcripts['transcript'].str.contains('YouTube is blocking requests from your IP', case=False, na=False)]
df_IP2.shape

In [ ]:
# function for fetching the data inc. the errors
def get_transcript(video_id, language='es'):
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=[language])
        return " ".join([line['text'] for line in transcript])
    except Exception as e:
        return f"Error: {e}"

In [ ]:
# applying the function,
for index, row in tqdm(df_IP2.iterrows(), total = 1421, desc="Retrying transcripts"):
    time.sleep(3)
    df_IP2.at[index, 'new_transcript'] = get_transcript(row['video_id'])

In [ ]:
df_IP2.head()

In [ ]:
#testing the performance
x = df_IP2.iloc[:60]
error_count = x[x['new_transcript'].str.startswith("Error", na=False)].shape[0]
print(f"Total transcripts with errors: {error_count}")
error_count = df_IP2[df_IP2['new_transcript'].str.startswith("Error", na=False)].shape[0]
print(f"Total transcripts with errors: {error_count}")

In [ ]:
# preparing the df to merge with the whole corpus
df_IP2 = df_IP2.drop(columns=['transcript'])
df_IP2 = df_IP2.rename(columns={'new_transcript': 'transcript'})  # Rename column

In [ ]:
df_IP2.to_csv("/df_IP2.csv")

In [ ]:
# Merging the two dataframes
all_transcripts.update(df_IP2)